In [2]:
import pandas as pd
import numpy as np

import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

In [3]:
calls_import = pd.read_csv('../data/Calls_Table_data.csv', delimiter='\t', encoding='utf-16')

In [38]:
import pandas as pd
import numpy as np
import datetime as dt

class DataRetrieval():
    
    #def __init__(self):

    
    def get_calls_data(self, filepath='../data/Calls_Table_data.csv', delimiter='\t'):
        """Retrieves call data from filepath"""
        df = pd.read_csv(filepath, delimiter='\t', encoding='utf-16')
        df.drop_duplicates(inplace=True)
        df.reset_index(drop=True, inplace=True)        
        return df
    
    
    
class FeaturizeCalls():
    """Clean incoming df to fit into model"""
    
    #def __init__(self):
    
    def fit(self, dataframe, y=None):
        self.dataframe = dataframe
        return self

    
    def transform(self):
        """tranform and clean incoming training or test"""
        df = self.dataframe.copy()      

        df['dt_time'] = pd.to_datetime(df['ORIG_TIME_QUEUED'])
        df['date'] = df['dt_time'].dt.date
        df['year'] = df['dt_time'].dt.year
        df['month'] = df['dt_time'].dt.month
        df['day'] = df['dt_time'].dt.day
        df['day_of_week'] = df['dt_time'].dt.weekday
        df['month_day'] = df['dt_time'].dt.strftime('%m/%d')
        df['month_weekday'] = df['dt_time'].dt.strftime('%b_%a')
        df['month'] = df['dt_time'].dt.strftime('%m/%d')        
        return df
    
    def make_holiday_dummies(self, holiday_dict=None, use_events=False):
        events = ({
            'Pride Parade' : ['6/30/2019', '6/24/2018', '6/25/2017', '6/26/2016', '6/28/2015',
                              '6/29/2014', '6/30/2013', '6/24/2012', '6/26/2011', '6/27/2010'],
            'Seafair' : ['8/2/2019', '8/3/2019', '8/4/2019', '8/3/2018', '8/4/2018', '8/5/2018',
                         '8/4/2017', '8/5/2017', '8/6/2017', '8/5/2016', '8/6/2016', '8/7/2016',
                         '7/31/2015', '8/1/2015', '8/2/2015', '8/1/2014', '8/2/2014', '8/3/2014',
                         '8/2/2013', '8/3/2013', '8/4/2013', '8/3/2012', '8/4/2012', '8/5/2012',
                         '8/5/2011', '8/6/2011', '8/7/2011', '8/6/2010', '8/7/2010', '8/8/2010' ],
            'Soltice Parade': ['6/30/2019', '6/16/2018', '6/17/2017', '6/18/2016', '6/20/2015',
                               '6/21/2014', '6/22/2013', '6/16/2012', '6/18/2011', '6/19/2010'],
            'Womens March' : ['1/19/2019', '1/20/2018', '1/21/2017'],})

        if use_events == True:
            holiday_dict = events
        _holidays = []
        for date in sorted(holiday_dict.keys()):
            for name in holiday_dict[date]:
                _holidays.append([date, name])
        _holidays = pd.DataFrame(_holidays, columns=['date', 'holiday'])
        return pd.get_dummies(_holidays.set_index('date')).reset_index()
    
    def add_dummies(self):
        pass

In [23]:
import pandas as pd
import numpy as np
import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

class SeattleHolidays:
    
    
    class CustomHolidays(holidays.US):
        def _populate(self, year=2019, start_year=2009, end_year=2030):
            # Populate the holiday list with the default US holidays
            holidays.US._populate(self, year)
            # Example: Add Ninja Turtle Day
            #self[dt.date(year, 7, 13)] = "Ninja Turtle Day"
            for year in range(start_year, end_year):
                # Add Valentine's day
                self[dt.date(year, 2, 14)] = "Valentines Day"
                # Add St Patricks Day
                self[dt.date(year, 3, 17)] = "St Patricks Day"
                # Add Easter
                self[holidays.easter(year=year)] = "Easter"
                # Add Good Friday
                self[holidays.easter(year=year)  -  dt.timedelta(days=2)] = "Good Friday"
                # Add Christmas Eve
                self[dt.date(year, 12, 24)] = "Christmas Eve"
                # Add New Years Eve
                self[dt.date(year, 12, 31)] = "New Years Eve"
                # Add Chinese New Year
                chinese = HongKong()
                for date, label in chinese.get_chinese_new_year(year):
                    self[date] = label


    class IslamicHolidays(holidays.HolidayBase):
        qatar_holidays = Qatar()
        def _populate(self, year=2019, start_year=2009, end_year=2030):
            # Populate the holiday list with blank base holidays
            holidays.HolidayBase._populate(self, year)
            for year in range(start_year, end_year):
                days = qatar_holidays.get_calendar_holidays(year)
                # Add Ramadan
                for i in range(1, len(days)):
                    if qatar_holidays.get_calendar_holidays(year)[i][1] == 'Start of ramadan':
                        for day in range(30):
                            self[qatar_holidays.get_calendar_holidays(year)[i][0] 
                                 + dt.timedelta(days=day-1)] = "Ramadan"
                    else:
                        self[qatar_holidays.get_calendar_holidays(2018)[1][0] 
                             - dt.timedelta(days=2)] = qatar_holidays.get_calendar_holidays(year)[i][1]


    class JewishHolidays(holidays.HolidayBase):
        def retrieve_data(self, filepath):
            df = pd.read_csv(filepath)
            return df

        def get_holidays(self, paths_list):
            df = self.retrieve_data(paths_list[0])
            for filepath in paths_list[1 : ]:
                cal = self.retrieve_data(filepath)
                df = pd.concat([df, cal])
            return df

        def _populate(self, year=2019, paths_list=['../data/hebcal_2010_usa.csv',
                                                   '../data/hebcal_2015_usa.csv', 
                                                   '../data/hebcal_2020_usa.csv',
                                                   '../data/hebcal_2025_usa.csv'],
                      start_year=2009, end_year=2030):
            hebcal = self.get_holidays(paths_list)
            hebcal.reset_index(drop=True, inplace=True)
            hebcal['date'] = pd.to_datetime(hebcal["Start Date"]).dt.date
            # Populate the holiday list with blank base holidays
            holidays.HolidayBase._populate(self, year)
            for year in range(start_year, end_year):
                for i in range(len(hebcal)):
                    self[hebcal['date'][i]] = hebcal['Subject'][i]


In [39]:
retriever = DataRetrieval()

In [40]:
df = retriever.get_calls_data()

In [41]:
cf = FeaturizeCalls()

In [42]:
cf.fit(df)

In [43]:
new_df = cf.transform()

In [44]:
holiday_test = SeattleHolidays.CustomHolidays()

In [45]:
holiday_test._populate()

In [46]:
cf.make_holiday_dummies(use_events=True)

,date,holiday_1/19/2019,holiday_1/20/2018,holiday_1/21/2017,holiday_6/16/2012,holiday_6/16/2018,holiday_6/17/2017,holiday_6/18/2011,holiday_6/18/2016,holiday_6/19/2010,...,holiday_8/5/2017,holiday_8/5/2018,holiday_8/6/2010,holiday_8/6/2011,holiday_8/6/2016,holiday_8/6/2017,holiday_8/7/2010,holiday_8/7/2011,holiday_8/7/2016,holiday_8/8/2010
0,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Pride Parade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
"""calls_full_df = calls_import

df.drop_duplicates(inplace=True)

df.reset_index(inplace=True)

df.columns

df['dt_time'] = pd.to_datetime(calls_full_df['ORIG_TIME_QUEUED'])

df['date'] = calls_full_df['dt_time'].dt.date

df['year'] = calls_full_df['dt_time'].dt.year

df['month'] = calls_full_df['dt_time'].dt.month

df['day'] = calls_full_df['dt_time'].dt.day

df['day_of_week'] = calls_full_df['dt_time'].dt.weekday

df['month_day'] = calls_full_df['dt_time'].dt.strftime('%m/%d')
    

df['month_weekday'] = calls_full_df['dt_time'].dt.strftime('%b_%a')

df['month'] = calls_full_df['dt_time'].dt.strftime('%m/%d')"""